# ingest.stores.dual

> Dual vector store implementation for ingesting documents into both sparse and dense stores

In [ ]:
# | default_exp ingest.stores.dual

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import os
from typing import List, Optional, Callable, Dict, Sequence, Union
from tqdm import tqdm
from langchain_core.documents import Document

from onprem.ingest.base import VectorStore
from onprem.ingest.stores.dense import DenseStore
from onprem.ingest.stores.sparse import SparseStore
from onprem.ingest.helpers import doc_from_dict

class DualStore(VectorStore):
    def __init__(
        self,
        dense_kind:str='chroma',
        dense_persist_directory: Optional[str] = None,
        sparse_kind:str='whoosh',
        sparse_persist_directory: Optional[str] = None,
        **kwargs
    ):
        """
        Initialize a dual vector store that manages both dense and sparse stores.
        
        **Args**:
        
          - *dense_persist_directory*: Path to dense vector database (created if it doesn't exist).
          - *sparse_persist_directory*: Path to sparse vector database (created if it doesn't exist).
          - *embedding_model_name*: name of sentence-transformers model
          - *embedding_model_kwargs*: arguments to embedding model (e.g., `{device':'cpu'}`). If None, GPU used if available.
          - *embedding_encode_kwargs*: arguments to encode method of embedding model (e.g., `{'normalize_embeddings': False}`).
        """
        self.init_embedding_model(**kwargs)  # stored in self.embeddings
        
        # Initialize both stores
        self.dense_store = DenseStore.create(
            kind=dense_kind,
            persist_directory=dense_persist_directory,
            embedding_model_name=kwargs.get('embedding_model_name'),
            embedding_model_kwargs=kwargs.get('embedding_model_kwargs'),
            embedding_encode_kwargs=kwargs.get('embedding_encode_kwargs')
        )
        self.sparse_store = SparseStore.create(
            kind=sparse_kind,
            persist_directory=sparse_persist_directory,
            embedding_model_name=kwargs.get('embedding_model_name'),
            embedding_model_kwargs=kwargs.get('embedding_model_kwargs'),
            embedding_encode_kwargs=kwargs.get('embedding_encode_kwargs')
        )
        
        # For compatibility with the VectorStore interface
        self.persist_directory = dense_persist_directory

    def keyword_search(self, query: str, **kwargs):
        """
        Perform keyword search using the sparse store.
        """
        return self.sparse_store.query(query, **kwargs)

    def get_dense_db(self):
        """
        Returns the dense store's database instance.
        """
        return self.dense_store.get_db()
    
    def get_sparse_db(self):
        """
        Returns the sparse store's database instance.
        """
        return self.sparse_store.get_db()


    #------------------------------
    # overrides of abstract methods
    # -----------------------------
  
    
    def exists(self):
        """
        Returns True if either store exists.
        """
        return self.dense_store.exists() or self.sparse_store.exists()
    
    def add_documents(self, documents: Sequence[Document], batch_size: int = 1000, **kwargs):
        """
        Add documents to both dense and sparse stores.
        """
        if not documents:
            return
        # Add to dense store
        self.dense_store.add_documents(documents, batch_size=batch_size, **kwargs)
        
        # Add to sparse store
        self.sparse_store.add_documents(documents, **kwargs)

   
    def remove_document(self, id_to_delete):
        """
        Remove a document from both stores.
        """
        self.dense_store.remove_document(id_to_delete)
        self.sparse_store.remove_document(id_to_delete)

    def remove_source(self, source:str):
        """
        Remove a document by source from both stores.

        The `source` can either be the full path to a document
        or a parent folder.  Returns the number of records deleted.
        """
        num_deleted_1 = self.dense_store.remove_source(source)
        num_deleted_2 = self.sparse_store.remove_source(source)
        return num_deleted_1
    

    def update_documents(self, doc_dicts: dict, **kwargs):
        """
        Update documents in both stores.
        """
        self.dense_store.update_documents(doc_dicts, **kwargs)
        self.sparse_store.update_documents(doc_dicts, **kwargs)
    
    def get_all_docs(self):
        """
        Get all documents from the dense store.
        For simplicity, we only return documents from one store since they should be the same.
        """
        return self.dense_store.get_all_docs()
    
    def get_doc(self, id):
        """
        Get a document by ID from the dense store.
        """
        return self.dense_store.get_doc(id)
    
    def get_size(self):
        """
        Get the size of the dense store.
        """
        return self.dense_store.get_size()
    
    def erase(self, confirm=True):
        """
        Erase both stores.
        """
        dense_erased = self.dense_store.erase(confirm=confirm)
        sparse_erased = self.sparse_store.erase(confirm=False)  # Second confirmation not needed
        return dense_erased and sparse_erased
    
    def query(self, q: str, **kwargs):
        """
        Query using the sparse store.
        """
        return self.sparse_store.query(q, **kwargs)
    
    def semantic_search(self, query: str, **kwargs):
        """
        Perform semantic search using the dense store.
        """
        return self.dense_store.semantic_search(query, **kwargs)
    
    def hybrid_search(self, query: str, limit: int = 10, weights: Union[List[float], float] = 0.5, **kwargs):
        """
        Perform hybrid search combining dense and sparse results.
        
        **Args**:
        - *query*: Search query string
        - *limit*: Maximum number of results to return
        - *weights*: Weights for combining dense and sparse scores. 
                    If float, represents dense weight (sparse = 1 - dense).
                    If list, [dense_weight, sparse_weight]
        - *kwargs*: Additional arguments passed to individual search methods
        
        **Returns**:
        Dictionary with 'hits' and 'total_hits' keys
        """
        # Create weights array if single number passed
        if isinstance(weights, (int, float)):
            weights = [weights, 1 - weights]
        
        # Get expanded results from both stores
        search_limit = limit * 10  # Get more candidates for better fusion
        
        # Get dense results
        dense_results = self.dense_store.query(query, limit=search_limit, return_dict=True, **kwargs)
        dense_hits = dense_results.get('hits', [])
        
        # Get sparse results  
        sparse_results = self.sparse_store.query(query, limit=search_limit, return_dict=True, **kwargs)
        sparse_hits = sparse_results.get('hits', [])
        
        # Combine scores using hybrid approach
        uids = {}
        
        # Process dense results (similarity-based scores)
        if weights[0] > 0:
            for dense_doc in dense_hits:
                uid = dense_doc.get('id')
                if uid:
                    score = dense_doc.get('score', 0.0)
                    uids[uid] = score * weights[0]
        
        # Process sparse results (use RRF since sparse doesn't have normalized scores)
        if weights[1] > 0:
            for rank, sparse_doc in enumerate(sparse_hits):
                uid = sparse_doc.get('id')
                if uid:
                    # Use Reciprocal Rank Fusion (RRF) for sparse results
                    rrf_score = 1.0 / (rank + 1)
                    if uid in uids:
                        uids[uid] += rrf_score * weights[1]
                    else:
                        uids[uid] = rrf_score * weights[1]
        
        # Sort by combined score and limit results
        sorted_results = sorted(uids.items(), key=lambda x: x[1], reverse=True)[:limit]
        
        # Convert back to result dictionaries
        results = []
        # Create a lookup dict for efficient document retrieval
        doc_lookup = {}
        for doc in dense_hits + sparse_hits:
            doc_lookup[doc.get('id')] = doc
        
        for uid, combined_score in sorted_results:
            if uid in doc_lookup:
                doc_dict = doc_lookup[uid].copy()
                doc_dict['score'] = combined_score  # Update with combined score
                results.append(doc_dict)
        
        return {'hits': results, 'total_hits': len(results)}

In [ ]:
show_doc(DualStore.exists)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dual.py#L75){target="_blank" style="float:right; font-size:smaller"}

### DualStore.exists

>      DualStore.exists ()

*Returns True if either store exists.*

In [ ]:
show_doc(DualStore.add_documents)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dual.py#L81){target="_blank" style="float:right; font-size:smaller"}

### DualStore.add_documents

>      DualStore.add_documents
>                               (documents:Sequence[langchain_core.documents.bas
>                               e.Document], batch_size:int=1000, **kwargs)

*Add documents to both dense and sparse stores.*

In [ ]:
show_doc(DualStore.remove_document)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dual.py#L93){target="_blank" style="float:right; font-size:smaller"}

### DualStore.remove_document

>      DualStore.remove_document (id_to_delete)

*Remove a document from both stores.*

In [ ]:
show_doc(DualStore.remove_source)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dual.py#L100){target="_blank" style="float:right; font-size:smaller"}

### DualStore.remove_source

>      DualStore.remove_source (source:str)

*Remove a document by source from both stores.*

In [ ]:
show_doc(DualStore.update_documents)

---

### DualStore.update_documents

>      DualStore.update_documents (doc_dicts:dict, **kwargs)

*Update documents in both stores.*

In [ ]:
show_doc(DualStore.get_all_docs)

---

### DualStore.get_all_docs

>      DualStore.get_all_docs ()

*Get all documents from the dense store.
For simplicity, we only return documents from one store since they should be the same.*

In [ ]:
show_doc(DualStore.get_doc)

---

### DualStore.get_doc

>      DualStore.get_doc (id)

*Get a document by ID from the dense store.*

In [ ]:
show_doc(DualStore.get_size)

---

### DualStore.get_size

>      DualStore.get_size ()

*Get the size of the dense store.*

In [ ]:
show_doc(DualStore.erase)

---

### DualStore.erase

>      DualStore.erase (confirm=True)

*Erase both stores.*

In [ ]:
show_doc(DualStore.query)

---

### DualStore.query

>      DualStore.query (q:str, **kwargs)

*Query using the dense store.*

In [ ]:
show_doc(DualStore.semantic_search)

---

### DualStore.semantic_search

>      DualStore.semantic_search (query:str, **kwargs)

*Perform semantic search using the dense store.*

In [ ]:
show_doc(DualStore.check)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/base.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### VectorStore.check

>      VectorStore.check ()

*Raise exception if `VectorStore.exists()` returns False*

In [ ]:
show_doc(DualStore.ingest)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/base.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### VectorStore.ingest

>      VectorStore.ingest (source_directory:str, chunk_size:int=500,
>                          chunk_overlap:int=50,
>                          ignore_fn:Optional[Callable]=None,
>                          batch_size:int=41000, **kwargs)

*Ingests all documents in `source_directory` (previously-ingested documents are
ignored). When retrieved, the
[Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)
objects will each have a `metadata` dict with the absolute path to the file
in `metadata["source"]`.
Extra kwargs fed to `ingest.load_single_document`.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_directory | str |  | path to folder containing document store |
| chunk_size | int | 500 | text is split to this many characters by [langchain.text_splitter.RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html) |
| chunk_overlap | int | 50 | character overlap between chunks in `langchain.text_splitter.RecursiveCharacterTextSplitter` |
| ignore_fn | Optional | None | Optional function that accepts the file path (including file name) as input and returns `True` if file path should not be ingested. |
| batch_size | int | 41000 | batch size used when processing documents |
| kwargs | VAR_KEYWORD |  |  |
| **Returns** | **None** |  |  |

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()